In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver   
from requests import get
import time
import re
import csv

In [2]:
url = "https://www.etherchain.org/blocks"
driver = webdriver.Chrome('/Users/wufangzheng/Downloads/chromedriver') #set the webdriver to Chrome driver
driver.get(url)                                                        #now we can use selenium to acces this website
time.sleep(10)                                                         #wait to let the selenium load the website

In [3]:
#find the very first block number on the page
element = driver.find_element_by_css_selector('tr.odd')     #find the first class named 'tr.odd'
content = element.find_element_by_tag_name('a') #in the class, find the first link which is the link of the block
link = content.get_attribute('href')            #get the link            
blockNumber = int(re.findall('\d{7}',link)[0])  #get the link number by regular expression
print(blockNumber)


9182088


In [4]:
tranlist = []

In [5]:

#loop for 100 blocks
for i in range(100):
    print('Now scrping block No.' + str(blockNumber) + ': ')
    
    #get in the website of each block
    while True:
        html = get('https://www.etherchain.org/block/' + str(blockNumber))
        if html.status_code == 200:
            print('Block Accessed!')
            break
        else:
            print('Access Denied, Reconnect in 5 seconds...')
            time.sleep(5)
    
    soup_object = BeautifulSoup(html.text,'html.parser')
    
    newstr = soup_object.findAll('td')     #all information we want is under the tag 'td'
    tID_set = re.findall('/tx/\w*', str(newstr))  #all transaction ID starts with '/tx/' and has 64 alphanumeric symbols
    
    #get in page of each transaction
    for tID in tID_set:
        tURL = 'https://www.etherchain.org' + tID  
        while True:
            tHTML = get(tURL)
            if tHTML.status_code == 200:
                break
            else:
                print('something wrong, reconnecting...')
                time.sleep(5)
        
        t_soup = BeautifulSoup(tHTML.text,'html.parser')
        t_str = t_soup.findAll('td') #all information we want is under the tag 'td'
    
    
        addresses = re.findall('/account/[\w]*', str(t_str)) #find string that starts with /account/
                                                         #first two occurences are addresses of sender and recepient
        
        value = re.findall('\d+[.]?[\d]* ETH', str(t_str)) #find the amount of transaction
                                                            #which has a form of '0 ETH' or 'xx.xxxx ETH'
        timestamp = re.findall('\d*[-]\d*[-]\d*[T]\d*[:]\d*[:]\d*', str(t_str)) #timestamp of the transaction
        print(timestamp)
        timestamp = re.split('T', str(timestamp[0]))
        
        #put all information of a transaction into a set
        info = []
        info.append(re.sub('/tx/', '0x', tID))
        info.append(re.sub('/account/', '0x', addresses[0]))
        info.append(re.sub('/account/', '0x', addresses[1]))
        info.append(value[0])
        info.append(timestamp[0])
        info.append(timestamp[1])
        tranlist.append(info)
        print('\nTransaction ID: ' + info[0] + ', From: ' + info[1] + ', To: ' + info[2] + ', Value: ' + info[3] + ', Date: ' + info[4] + ', Time: ' + info[5])
        
    blockNumber-=1
    

Now scrping block No.9182088: 
Block Accessed!
['2019-12-29T18:26:26']

Transaction ID: 0xf4a118d8c9c306bc12ab45ab0d4ee58481cb288027be8dfc8a6a15acf0769ac4, From: 0x2d53c98aee1bb04337f8e32f2c4cb9aa2b2ac1e3, To: 0xdb2b3db44820526e7472dea06506b2e40506b3b9, Value: 0 ETH, Date: 2019-12-29, Time: 18:26:26
['2019-12-29T18:26:26']

Transaction ID: 0x6f9c36c0042295e5e928791efdb981d0d2eaacd0bcea21ecd15c137e2ad00a86, From: 0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5, To: 0x16db2ff40df3525a8bc551ae49014bb922391f29, Value: 0.05004 ETH, Date: 2019-12-29, Time: 18:26:26
['2019-12-29T18:26:26']

Transaction ID: 0x6a74a11a72e30fc56e52a55703d13af83ef6c4cf379528c4b24364ee3f28cfd7, From: 0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5, To: 0xd30da56d282d682455fa88b0cf41fa3e3acfe74d, Value: 0.20019 ETH, Date: 2019-12-29, Time: 18:26:26
['2019-12-29T18:26:26']

Transaction ID: 0x31bbc469db3b8e57d1e5af6279dd975f0d869237a459a19d47ceb882ea44c152, From: 0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5, To: 0xb6c6ccc5e9ce0e

['2019-12-29T18:26:26']

Transaction ID: 0xd9e031bdd6bfcf21eb2ee749a3b033a27301cc1a580997ce8f127b7442b8688f, From: 0xfe79f8d7c0104bb9ab61f8b43c9c204c444eb2b0, To: 0xd7cc16500d0b0ac3d0ba156a584865a43b0b0050, Value: 0 ETH, Date: 2019-12-29, Time: 18:26:26
['2019-12-29T18:26:26']

Transaction ID: 0x9eab29559bbf1f7c7553d9911273d3defbfe17799c0448afe29d02c7b9734004, From: 0x719b8be8e2c3f07575d41a2580011db702029e5b, To: 0x164c93580839f40609ce0250dd4c98a25da175de, Value: 0.001 ETH, Date: 2019-12-29, Time: 18:26:26
['2019-12-29T18:26:26']

Transaction ID: 0xdadf362631225b08d4ebeb2e76b7ba81931c6cad0fa9abadbae05ae1de2da417, From: 0xdab6d4e7852c5d823be1ce0db79c08a38f3b6d81, To: 0xed6b0dc3aa8de5908ab857a70cb2ff657d9b6c5d, Value: 0 ETH, Date: 2019-12-29, Time: 18:26:26
['2019-12-29T18:26:26']

Transaction ID: 0x2f0667b3e6a52a37f225c2a36639269653179ad8743964a0b11bf3f1b8ad09b7, From: 0x00f0360e443579714a0319fd72ab75ff07af29c1, To: 0xf5ae0e27cf423b1ed6513c3ca35cc14c22ee66fd, Value: 0 ETH, Date: 2019-12

['2019-12-29T18:26:26']

Transaction ID: 0x7f99e1036e749c4f94805a424bcde8da8fbb447343861b7d0983b2844e630284, From: 0x00192fb10df37c9fb26829eb2cc623cd1bf599e8, To: 0xf14f6b1ba78d1a4532474c7a92030419e042f34f, Value: 0.05007 ETH, Date: 2019-12-29, Time: 18:26:26
['2019-12-29T18:26:26']

Transaction ID: 0xa86f3215e58cd4206dd5c0ab339025cfee858e7700b14fbb4fbc2fe3b0afb584, From: 0x00192fb10df37c9fb26829eb2cc623cd1bf599e8, To: 0x359d01af789cfc57907e838724e796cbd486ba10, Value: 0.06233 ETH, Date: 2019-12-29, Time: 18:26:26
['2019-12-29T18:26:26']

Transaction ID: 0x6af8771938f806c02ea2eb2c049507ea10cc3b2b9b5c3452bc8e6fb1d46ac248, From: 0x00192fb10df37c9fb26829eb2cc623cd1bf599e8, To: 0x7bcba1f615f82406ed15d4f023a6eff54451171a, Value: 0.22705 ETH, Date: 2019-12-29, Time: 18:26:26
something wrong, reconnecting...
something wrong, reconnecting...
something wrong, reconnecting...
['2019-12-29T18:26:26']

Transaction ID: 0xa769019585240258f84313a37842cac93f9c895262fd21fa9907b6589445c9ef, From: 0x0019

KeyboardInterrupt: 

In [6]:
with open('Ether.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows([['Transaction ID', 'From', 'To', 'Value', 'Date', 'Time']])
    writer.writerows(tranlist)